In [185]:
using GuSTO
using AstrobeeRobot

In [186]:
robot = Astrobee3D()
model = AstrobeeSE3Manifold()
env = ISSCorner();
add_obstacles1!(env)

x_init = [11.2; -0.8; 5.6]
x_goal = [10.2; 6.9; 4.2]

x_init = [x_init; zeros(3); sqrt(1/3)*[1.; 0.; 1.; 1.]; zeros(3)]
x_goal = [x_goal; zeros(3); [-0.5; 0.5; -0.5; 0.5]; zeros(3)]
# q_goal = [0.01; 0.01; 0.01; 1.]
# q_goal = -q_goal/norm(q_goal)/
# x_init = [x_init; zeros(3); [0.; 0.; 0.; 1.]; zeros(3)]
# x_goal = [x_goal; zeros(3); [0.; 0.; 0.; 1.]; zeros(3)]

N = 100
tf_guess = 70.

PD = ProblemDefinition(robot, model, env, x_init, x_goal);
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
# solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_jump!, init_traj_straightline, "IPOPT", print_level=0)


LoadError: [91mUnexpected object SCPVariables{JuMP.Variable,Array{JuMP.Variable,N} where N}(X[i,j] ∀ i ∈ {1,2,…,12,13}, j ∈ {1,2,…,99,100}, U[i,j] ∀ i ∈ {1,2,…,5,6}, j ∈ {1,2,…,99,100}, Tf) in nonlinear expression.[39m

In [187]:
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
;

TOSgusto.SCPS.converged = false
TOSgusto.SCPS.iterations = 0
TOSgusto.SCPS.total_time = 0.0
TOSgusto.SCPS.accept_solution = Bool[true]
TOSgusto.SCPS.prob_status = Symbol[:NA]
TOSgusto.SCPS.convergence_measure = [0.0]
TOSgusto.SCPS.param.alg.omega_vec = [1.0]
TOSgusto.SCPS.param.alg.Delta_vec = [10.0]


In [6]:
plot(TOSgusto.SCPS.traj.X[13,:])

LoadError: [91mUndefVarError: TOSgusto not defined[39m

In [7]:
# Quaternion norm
normq = [norm(TOSgusto.SCPS.traj.X[7:10,i]) for i in 1:TOSgusto.SCPS.SCPP.N]
plot(normq)

LoadError: [91mUndefVarError: TOSgusto not defined[39m

In [8]:
# Animate Astrobee trajectory
vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

speed_factor = 1

Qs = Vector{Vector{Float64}}(0)
for k in 1:speed_factor:N
    q = [quat_inv(TOSgusto.SCPS.traj.X[7:10,k]); TOSgusto.SCPS.traj.X[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)

LoadError: [91mUndefVarError: env not defined[39m